# Environment

In [2]:
!pip3 install mlcroissant
!pip3 install rdflib
!pip3 install pandas
import mlcroissant as mlc
from rdflib import Graph, Namespace, URIRef, Literal
import json
import pandas as pd

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.4/562.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00
  Created wheel for jsonpath-rw: filename=jsonpath_rw-1.4.0-py3-none-any.whl size=15130 sha256=5cc6269060d32e134dd58c6f7b30e8d3635a7a4a581bc86a938f0a7c24afa750
  Stored in directory: /root/.cache/pip/wheels/e3/76/6f/c25be6a9e6cc9985b96e8c95997d46790242c6426ef68e754c
Successfully built jsonpath-rw


# Load HuggingFace RDF

In [3]:
# Smaller dataset
g = Graph().parse('https://github.com/david4096/huggingface-rdf/releases/download/data-release/huggingface.ttl?download=true', format='ttl')
# Larger dataset
# g = Graph().parse('https://huggingface.co/datasets/david4096/huggingface-ttl/resolve/main/huggingface-30k.ttl?download=true', format='ttl')


# Queries

Query to get property counts by dataset

List of classes: (to be completed)

*   https://schema.org/Dataset
*   http://mlcommons.org/croissant/RecordSet

In [20]:
# @title Query number of each property of each class
class_URI = "https://schema.org/Dataset" # @param {"type":"string","placeholder":"https://schema.org/Dataset"}
query = """


SELECT ?dataset ?dataset_name ?predicate (count(?o) as ?predicate_count)
WHERE {
    ?dataset <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <"""+class_URI+"""> .
    ?dataset ?predicate ?o .
    OPTIONAL { ?dataset <https://schema.org/name> ?dataset_name . }
}
GROUP BY ?predicate ?dataset
"""
print(query)

qres = g.query(query)
# for row in qres:
#     print(f"Dataset: {row.dataset_name} Predicate: {row.predicate} Count: {row.predicate_count}")




SELECT ?dataset ?dataset_name ?predicate (count(?o) as ?predicate_count)
WHERE {
    ?dataset <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://schema.org/Dataset> .
    ?dataset ?predicate ?o .
    OPTIONAL { ?dataset <https://schema.org/name> ?dataset_name . }
}
GROUP BY ?predicate ?dataset



# Export data

Export to TSV

In [21]:
with open("query_results.tsv", "w") as outfile:
  first_row = True
  for row in qres:
    if first_row:
      for label in row.labels.keys():
        outfile.write(label + "\t")
      outfile.write("\n")
      first_row = False
    for value in row:
      outfile.write(value + "\t")
    outfile.write("\n")

In [22]:
df = pd.read_csv("query_results.tsv", sep = "\t")

In [23]:
print(df['predicate'].value_counts())

predicate
http://www.w3.org/1999/02/22-rdf-syntax-ns#type    2428
http://purl.org/dc/terms/conformsTo                2428
https://schema.org/alternateName                   2428
https://schema.org/creator                         2428
https://schema.org/description                     2428
https://schema.org/distribution                    2428
https://schema.org/keywords                        2428
https://schema.org/name                            2428
https://schema.org/url                             2428
http://mlcommons.org/croissant/recordSet           2208
https://schema.org/sameAs                           713
https://schema.org/license                          695
https://schema.org/identifier                         5
Name: count, dtype: int64


# Archive

# Generate ttl

In [ ]:
urls = ["https://huggingface.co/api/datasets/fashion_mnist/croissant", "https://huggingface.co/api/datasets/abisee/cnn_dailymail/croissant"]
baseurl = "http://example.org/"
g = Graph()

for url in urls:
  json_metadata = mlc.Dataset(url).metadata.to_json()

  g.parse(
      data=json.dumps(json_metadata),
      format='json-ld',
      base=URIRef(baseurl)
  )

print(g.serialize(format='ttl'))